In [1]:
import itertools
import warnings

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.stats as st
import scipy.special

# The MCMC Hammer
import emcee

import numba

# BE/Bi 103 utilities
import bebi103

# Import plotting tools
import matplotlib.pyplot as plt
import seaborn as sns
import corner

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# This enables high res graphics inline (only use with static plots (non-Bokeh))
# SVG is preferred, but there is a bug in Jupyter with vertical lines
%config InlineBackend.figure_formats = {'png', 'retina'}

# JB's favorite Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df = pd.read_csv("./data/gardner_hw6/gardner_mt_catastrophe_only_tubulin.csv", comment = "#")

In [3]:
df.head()

,12 uM,7 uM,9 uM,10 uM,14 uM
0,25.000,35,25,50,60
1,40.000,45,40,60,75
2,40.000,50,40,60,75
3,45.429,50,45,75,85
4,50.000,55,50,75,115


In [4]:
tau = np.array([15,10,17])
t=100

#m=0
for i in range(2):
    if not tau[i]<tau[i+1]:
        sum_z = -np.inf
        break
        
    else:
        x=[] #d
        y=[] #num
        z=[] #function output
        x_array_product=[] #den

        for j in range(3):
            y.append( (tau[j]*np.exp(-t/tau[j]) ))

            for k in range(3):
                if k != j:
                    x.append(tau[j] - tau[k])
            x_array=np.array(x)
            x_array_product.append(np.prod(x_array))

            z.append(y[j]/x_array_product[j])
            x=[]
            #m+=1
        z_array=np.array(z)
        if np.sum(z)>0:
            sum_z = np.log(np.sum(z))
        else:
            RaiseError("WTF man!?")
    break
            

In [31]:
def log_likelihood(tau, t, m):
    """
    Takes in tau (should be an array), 
    data, various values for m. We are very proud of this function.
    """
    tau = np.sort(tau)
    
    num_handler = [] #numerator
    den_handler = [] #denominator
    den_elements = [] #elements necessary to take products in den
    func_handler = [] #output log posterior
    
    for i in range(m-1):

        if not tau[i]<tau[i+1]:

            return -np.inf
            
        for j in range(m):
            num_handler.append(tau[j]**(m-2) * np.exp(-t/tau[j]))

            for k in range(m):
                if k != j:
                    den_elements.append(tau[j] - tau[k])

            #den_elements_array = np.array(den_elements)

            den_handler.append(np.prod(den_elements))

            func_handler.append(num_handler[j]/den_handler[j])

            den_elements = []

        #func_handler_array = np.array(func_handler)

    if np.sum(func_handler)>0:
        return np.log(np.sum(func_handler))

    else:
        #raise RuntimeError("WTF Man.")
        return -np.inf

def log_prior(tau, m):
    """
    Log prior for model defined above. Takes in m and tau, 
    returns log prior.
    """
    tau = np.sort(tau)
    
    for i in range(m-1):

        if not tau[i]<tau[i+1]:

            return -np.inf
    
    if not np.all(tau> 0):
        return -np.inf
    
    return -np.log(np.prod(tau))
        
test_run = log_likelihood(tau, t, 3)

def tau_start(m, n_walkers, n_temps):
    p = np.empty((n_temps, n_walkers, m))
    
    for i in range(m):
        p[:,:,i]=np.random.exponential(1, (n_temps,n_walkers))
    return p


In [10]:
df["12 uM"].head()

0    25.000
1    40.000
2    40.000
3    45.429
4    50.000
Name: 12 uM, dtype: float64

In [11]:
n_burn=100
n_steps=1000
n_temps=20
p0=tau_start(3,50,20)
loglargs = (df["12 uM"], 3)
logpargs = (3,)
columns = ["tau_1", "tau_2", "tau_3"]

In [32]:
df_out, lnZ_out, dlnZ_out = bebi103.run_pt_emcee(log_likelihood, log_prior, n_burn, n_steps, 
                    n_temps = n_temps, p0 = p0, loglargs = loglargs, 
                    logpargs = logpargs, threads = None, columns = columns,
                    return_lnZ=True)

Traceback (most recent call last):
  File "//anaconda/lib/python3.4/site-packages/IPython/core/interactiveshell.py", line 3066, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-32-522b574afdb9>", line 4, in <module>
    return_lnZ=True)
  File "/Users/Shashank/Desktop/Caltech/GitHub/DataAnalysisHomeworkRepository/Homework6/bebi103.py", line 560, in run_pt_emcee
    _ = sampler.run_mcmc(pos, n_steps)
  File "//anaconda/lib/python3.4/site-packages/emcee/sampler.py", line 157, in run_mcmc
    **kwargs):
  File "//anaconda/lib/python3.4/site-packages/emcee/ptsampler.py", line 322, in sample
    results = list(map(fn, qs.reshape((-1, self.dim))))
  File "//anaconda/lib/python3.4/site-packages/emcee/ptsampler.py", line 96, in __call__
    return self.logl(x, *self.loglargs, **self.loglkwargs), lp
  File "<ipython-input-31-c34f8843ae27>", line 20, in log_likelihood
    num_handler.append(tau[j]**(m-2) * np.exp(-t/tau[j]))
  File "//anaconda/lib/python3.

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable

In [23]:
t

100

In [ ]:
x=[1,2,3]
np.prod(x)

In [13]:
a, b, c=5, 3, 2

In [14]:
np.log(np.exp(a) - np.exp(b) + np.exp(c))

4.910569464803074

In [15]:
a + np.log(1 - np.exp(b-a) + np.exp(c-a))

4.910569464803074

In [24]:
t = df["12 uM"][0]

In [33]:
df_out

NameError: name 'df_out' is not defined

In [27]:
tau = np.array([0.06463911, 0.04257534, 0.11927362])
z=log_likelihood(tau,t,3)

In [28]:
z

-205.56031209255227